# Playground for Amazon SES

This uses the `boto` package - which is not the same as `boto3`...

In [1]:
# !pip install boto

In [2]:
import boto.ses

In [3]:
import awskeys
AWS_ACCESS_KEY = awskeys.AWS_ACCESS_KEY
AWS_SECRET_KEY = awskeys.AWS_SECRET_KEY
AWS_MY_EMAIL1   = awskeys.AWS_MY_EMAIL1       # Work e-mail
AWS_MY_EMAIL2   = awskeys.AWS_MY_EMAIL2       # G-Mail account

In [8]:
class Email(object):  
    def __init__(self, to, subject):
        self.to = to
        self.subject = subject
        self._html = None
        self._text = None
        self._format = 'html'

    def html(self, html):
        self._html = html

    def text(self, text):
        self._text = text

    def send(self, from_addr=None):
        body = self._html

        if isinstance(self.to, str):
            self.to = [self.to]
        if not from_addr:
            from_addr = 'me@example.com'
        if not self._html and not self._text:
            raise Exception('You must provide a text or html body.')
        if not self._html:
            self._format = 'text'
            body = self._text

        connection = boto.ses.connect_to_region(
            'eu-west-1',
            aws_access_key_id=AWS_ACCESS_KEY, 
            aws_secret_access_key=AWS_SECRET_KEY
        )

        #print ('From: ' + from_addr + ' to ' + str(self.to))
        
        return connection.send_email(
            from_addr,
            self.subject,
            None,
            self.to,
            format=self._format,
            text_body=self._text,
            html_body=self._html
        )

In [9]:
# Test sending an e-mail
email = Email(to=AWS_MY_EMAIL1, subject='Test-email TXT')  
email.text('This is a text body. Test.')  
email.send(from_addr=AWS_MY_EMAIL1).MessageId

'0102016519dd2134-f499897e-8571-4715-9746-54c1cfe69f4b-000000'

In [10]:
# Some more examples
email = Email(to=AWS_MY_EMAIL1, subject='Test-email HTML')  
email.html('<html><body>This is an HTML text body. <strong>Test.</strong></body></html>')  
email.send(from_addr=AWS_MY_EMAIL1)

email = Email(to=AWS_MY_EMAIL2, subject='Test-email TXT')  
email.text('This is a text body. Test.')  
email.send(from_addr=AWS_MY_EMAIL1)

email = Email(to=AWS_MY_EMAIL2, subject='Test-email HTML')  
email.html('<html><body>This is an HTML text body. <strong>Test.</strong></body></html>')  
email.send(from_addr=AWS_MY_EMAIL1)  

email = Email(to=AWS_MY_EMAIL1, subject='Test-email HTML')  
email.html('<html><body>This is an HTML text body. <strong>Test.</strong></body></html>')  
email.send(from_addr=AWS_MY_EMAIL2)

email = Email(to=AWS_MY_EMAIL1, subject='Test-email HTML')  
email.text('This is a text body. Testing whether from/to need to be different - this should work.')  
email.send(from_addr=AWS_MY_EMAIL2)

email = Email(to=AWS_MY_EMAIL2, subject='Test-email HTML')  
email.text('This is a text body. Testing whether from/to need to be different - this should *not* work.')  
email.send(from_addr=AWS_MY_EMAIL2) 

{'SendEmailResponse': {'ResponseMetadata': {'RequestId': '63299219-9b14-11e8-b04f-91752afb6e8f'},
  'SendEmailResult': {'MessageId': '0102016519dd2d20-b41e63bd-034e-4d8d-a2bd-f8b1fe8eab5b-000000'}}}